In [2]:
import pandas as pd
import numpy as np

# --- Load data ---
df = pd.read_excel("SP500Index.xlsx",
                   dtype={"Date of Observation": str})  # ensure no loss of leading zeros

# --- Rename and parse dates (format YYYYMMDD) ---
df = df.rename(columns={
    "Date of Observation": "date",
    "Level of the S&P 500 Index": "SP"
})

# Parse the yyyymmdd strings to datetime
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d", errors="coerce")

# Clean + sort
df = df.dropna(subset=["date", "SP"]).sort_values("date").reset_index(drop=True)

# If your workbook contains data before Jan-1960, keep only from 1960-01-01:
# df = df[df["date"] >= pd.Timestamp("1960-01-01")].reset_index(drop=True)

# --- Build x_t = log(SP_t / SP_0), with t=0 at the first row (Jan 1960) ---
sp0 = df.loc[0, "SP"]
df["x"] = np.log(df["SP"] / sp0)

# --- Increments Δx_t and MLEs ---
dx = df["x"].diff().dropna()  # Δx_t = x_t - x_{t-1}, t=1..T
T = int(dx.shape[0])

delta_hat = dx.mean()
sigma2_hat = ((dx - delta_hat)**2).mean()   # MLE with denominator T
sigma_hat = float(np.sqrt(sigma2_hat))

# --- Annualise (assuming monthly data) ---
delta_hat_annual = 12 * delta_hat
sigma_hat_annual = np.sqrt(12) * sigma_hat

print("Sample runs from", df.loc[0, "date"].date(), "to", df.loc[len(df)-1, "date"].date())
print(f"Observations (T): {T}")
print(f"Monthly MLE drift   δ̂  : {delta_hat: .6f}")
print(f"Monthly MLE vol     σ̂  : {sigma_hat: .6f}")
print(f"Annualised drift  12·δ̂ : {delta_hat_annual: .6f}")
print(f"Annualised vol   √12·σ̂ : {sigma_hat_annual: .6f}")


Sample runs from 1960-01-29 to 2018-06-29
Observations (T): 701
Monthly MLE drift   δ̂  :  0.005548
Monthly MLE vol     σ̂  :  0.042180
Annualised drift  12·δ̂ :  0.066581
Annualised vol   √12·σ̂ :  0.146117
